# Lesson 6: 모델 평가

Deeplearning.AI & Upstage의 다음 강의를 듣고 정리한 노트북입니다.

https://learn.deeplearning.ai/courses/pretraining-llms/lesson/7/evaluation

이 노트북에서는 pretrained LLM의 성능을 평가하는 방법에 대해 학습합니다. 모델의 평가는 loss추이/직접 확인/모델 비교/벤치마크 등으로 수행합니다.

1. loss추이 : 학습이 진행됨에 따라 loss가 감소하는지 확인
2. 직접확인 : 일정 스텝마다 실제 산출 결과를 통해 개선되는지 확인
3. 모델 비교 : 동일한 프롬프트에 대해 다른 모델과의 산출물 비교
4. 벤치마크 : 여러 알려진 벤치마크를 수행하여 성능 비교

LLM 벤치마크를 수행할 수 있는 유용한 툴 중 하나로 EleutherAI에서 만든 __[LM Evaluation Harness](https://github.com/EleutherAI/lm-evaluation-harness)__ 를 소개합니다. 

In [ ]:
# Ignore insignificant warnings (ex: deprecation warnings)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#!pip install -U git+https://github.com/EleutherAI/lm-evaluation-harness

이 툴을 이용하여 `TinySolar-248m-4k` 모델의 __[TrustfulQA MC2 task](https://arxiv.org/abs/2109.07958)__ 에 대한 성능을 측정합니다. 이는 선다형 문제(multiple-choice question)의 답변을 얼마나 잘 찾아내는지를 측정하는 태스크입니다.

In [4]:
!lm_eval --model hf \
    --model_args pretrained="upstage/TinySolar-248m-4k" \
    --tasks truthfulqa_mc2 \
    --device cpu \
    --limit 5

2024-07-23:14:50:43,991 INFO     [__main__.py:272] Verbosity set to INFO
2024-07-23:14:50:44,039 INFO     [__init__.py:491] `group` and `group_alias` keys in tasks' configs will no longer be used in the next release of lm-eval. `tag` will be used to allow to call a collection of tasks just like `group`. `group` will be removed in order to not cause confusion with the new ConfigurableGroup which will be the offical way to create groups with addition of group-wide configuations.
2024-07-23:14:50:47,564 INFO     [__init__.py:512] The tag xnli is already registered as a group, this tag will not be registered. This may affect tasks you want to call.
2024-07-23:14:50:47,566 INFO     [__init__.py:512] The tag xnli is already registered as a group, this tag will not be registered. This may affect tasks you want to call.
2024-07-23:14:50:47,567 INFO     [__init__.py:512] The tag xnli is already registered as a group, this tag will not be registered. This may affect tasks you want to call.
2024-

## HuggingFace 리더보드 평가

어래 코드를 통해서 각자 개발한 모델을 [HuggingFace Leaderboard](https://huggingface.co/open-llm-leaderboard)에 오르기 위해 필요한 벤치마크를 수행할 수 있습니다. 

In [ ]:
import os

def h6_open_llm_leaderboard(model_name):
  task_and_shot = [
      ('arc_challenge', 25),
      ('hellaswag', 10),
      ('mmlu', 5),
      ('truthfulqa_mc2', 0),
      ('winogrande', 5),
      ('gsm8k', 5)
  ]

  for task, fewshot in task_and_shot:
    eval_cmd = f"""
    lm_eval --model hf \
        --model_args pretrained={model_name} \
        --tasks {task} \
        --device cpu \
        --num_fewshot {fewshot}
    """
    os.system(eval_cmd)

h6_open_llm_leaderboard(model_name="MODEL")